## 1. Setup and Imports

In [ ]:
# Install required packages (run once)
# !pip install monai nibabel torch torchvision scikit-learn matplotlib tqdm

In [ ]:
import os
import json
import numpy as np
import nibabel as nib
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from monai.networks.nets import SEResNet50
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Configuration

In [ ]:
# ============== MODIFY THESE PATHS ==============
# Path to straightened CT volumes (whole spine, vertebra-centered)
CT_FOLDER = "/content/verse19/straighten/CT"  # Colab path
# CT_FOLDER = "d:/Graduation Project/HeathiVert/verse19/straighten/CT"  # Windows path

# Path to mask_2d folder (binary masks for target vertebra localization)
MASK_FOLDER = "/content/verse19/straighten/mask_2d"  # Colab path
# MASK_FOLDER = "d:/Graduation Project/HeathiVert/verse19/straighten/mask_2d"  # Windows path

# Path to vertebra_data.json with ground truth labels
JSON_PATH = "/content/vertebra_data.json"  # Colab path  
# JSON_PATH = "d:/Graduation Project/HeathiVert/vertebra_data.json"  # Windows path

# Output folder for checkpoints
CHECKPOINT_FOLDER = "/content/checkpoints/classifier_localized"  # Colab path
# CHECKPOINT_FOLDER = "d:/Graduation Project/HeathiVert/checkpoints/classifier_localized"  # Windows path

# ============== TRAINING HYPERPARAMETERS ==============
BATCH_SIZE = 32
NUM_EPOCHS = 50
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
NUM_SLICES = 30  # Number of slices to extract from each volume
VAL_SPLIT = 0.2  # Validation split ratio
CROP_SIZE = 128  # Size to crop around vertebra bounding box

# Create checkpoint folder
os.makedirs(CHECKPOINT_FOLDER, exist_ok=True)
print(f"Checkpoints will be saved to: {CHECKPOINT_FOLDER}")
print(f"\nAPPROACH: WITH MASK LOCALIZATION (crop {CROP_SIZE}x{CROP_SIZE} around target)")

## 3. Load Ground Truth Labels

The JSON structure is:
```json
{
  "train": {"sub-verse004_23": 0, "sub-verse020_12": 1, ...},
  "test": {...},
  "val": {...}
}
```

We convert Genant grades (0,1,2,3) to binary (0=healthy, 1=fractured)

In [ ]:
def load_labels(json_path):
    """Load vertebra labels from JSON and convert to binary classification."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Combine all splits
    all_labels = {}
    for split in ['train', 'test', 'val']:
        if split in data:
            all_labels.update(data[split])
    
    # Convert to binary: 0 = healthy, 1+ = fractured
    binary_labels = {k: (1 if v > 0 else 0) for k, v in all_labels.items()}
    
    # Statistics
    n_healthy = sum(1 for v in binary_labels.values() if v == 0)
    n_fractured = sum(1 for v in binary_labels.values() if v == 1)
    
    print(f"Total vertebrae: {len(binary_labels)}")
    print(f"  Healthy (0): {n_healthy} ({100*n_healthy/len(binary_labels):.1f}%)")
    print(f"  Fractured (1): {n_fractured} ({100*n_fractured/len(binary_labels):.1f}%)")
    
    return binary_labels

labels = load_labels(JSON_PATH)

## 4. Dataset Class WITH Mask-Based Localization

**Key Points:**
- Each CT file contains the **whole straightened spine** (multiple vertebrae)
- The `mask_2d` folder contains a **binary mask** isolating only the target vertebra
- We find the **bounding box** of the target vertebra from the mask
- Extract 2D slices **cropped around the vertebra center** (128×128)
- This ensures the classifier sees ONLY ONE vertebra per sample

In [ ]:
class VertebraSliceDatasetLocalized(Dataset):
    """Dataset that extracts 2D slices using mask_2d for localization.
    
    Uses binary mask to find the target vertebra bounding box and crops
    a fixed-size region around it. This ensures the model sees ONLY the
    target vertebra, not neighboring ones.
    """
    
    def __init__(self, ct_folder, mask_folder, labels_dict, num_slices=30, crop_size=128, transform=None):
        """
        Args:
            ct_folder: Path to folder containing CT .nii.gz files (whole spine)
            mask_folder: Path to folder containing mask_2d .nii.gz files (target vertebra mask)
            labels_dict: Dictionary mapping vertebra_id -> binary label
            num_slices: Number of slices to extract from each volume
            crop_size: Size of the square crop around vertebra center
            transform: Optional transforms to apply to each slice
        """
        self.ct_folder = Path(ct_folder)
        self.mask_folder = Path(mask_folder)
        self.labels_dict = labels_dict
        self.num_slices = num_slices
        self.crop_size = crop_size
        self.transform = transform
        
        # Cache for bounding boxes to avoid recomputing
        self.bbox_cache = {}
        
        # Find all .nii.gz files and their labels
        self.samples = []  # List of (ct_path, mask_path, slice_idx, label)
        self._prepare_samples()
        
    def _get_vertebra_bbox(self, mask_path):
        """Get bounding box of vertebra from mask. Returns (y_min, y_max, x_min, x_max, z_min, z_max)."""
        if str(mask_path) in self.bbox_cache:
            return self.bbox_cache[str(mask_path)]
        
        mask = nib.load(str(mask_path)).get_fdata()
        # Find non-zero voxels
        nonzero = np.where(mask > 0)
        
        if len(nonzero[0]) == 0:
            return None
        
        bbox = (
            nonzero[0].min(), nonzero[0].max(),  # y_min, y_max
            nonzero[1].min(), nonzero[1].max(),  # x_min, x_max
            nonzero[2].min(), nonzero[2].max()   # z_min, z_max
        )
        self.bbox_cache[str(mask_path)] = bbox
        return bbox
        
    def _prepare_samples(self):
        """Build list of (ct_path, mask_path, slice_index, label) tuples."""
        ct_files = list(self.ct_folder.glob('*.nii.gz'))
        print(f"Found {len(ct_files)} CT .nii.gz files")
        
        matched = 0
        unmatched = []
        
        for ct_path in ct_files:
            # Extract vertebra ID from filename
            filename = ct_path.stem.replace('.nii', '')
            
            # Try different ID formats for matching with JSON
            possible_ids = [
                filename,                          # sub-verse004_ct_23
                filename.replace('_ct_', '_'),     # sub-verse004_23
            ]
            
            label = None
            for pid in possible_ids:
                if pid in self.labels_dict:
                    label = self.labels_dict[pid]
                    break
            
            if label is not None:
                # Check if corresponding mask exists
                mask_path = self.mask_folder / ct_path.name
                if not mask_path.exists():
                    print(f"Warning: Mask not found for {ct_path.name}")
                    continue
                
                # Get bounding box from mask
                bbox = self._get_vertebra_bbox(mask_path)
                if bbox is None:
                    print(f"Warning: Empty mask for {ct_path.name}")
                    continue
                
                matched += 1
                z_min, z_max = bbox[4], bbox[5]
                z_center = (z_min + z_max) // 2
                
                # Extract slices centered on vertebra's z-range
                half_slices = self.num_slices // 2
                start = max(z_min, z_center - half_slices)
                end = min(z_max + 1, z_center + half_slices)
                
                for slice_idx in range(start, end):
                    self.samples.append((ct_path, mask_path, slice_idx, label))
            else:
                unmatched.append(filename)
        
        print(f"Matched: {matched} vertebrae")
        print(f"Unmatched: {len(unmatched)} vertebrae")
        print(f"Total slices: {len(self.samples)}")
        
        # Class distribution
        n_healthy = sum(1 for _, _, _, l in self.samples if l == 0)
        n_fractured = sum(1 for _, _, _, l in self.samples if l == 1)
        print(f"Slice distribution: Healthy={n_healthy}, Fractured={n_fractured}")
        
        if unmatched[:5]:
            print(f"Sample unmatched IDs: {unmatched[:5]}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        ct_path, mask_path, slice_idx, label = self.samples[idx]
        
        # Load CT volume and extract slice
        ct_vol = nib.load(str(ct_path)).get_fdata()
        slice_2d = ct_vol[:, :, slice_idx].astype(np.float32)
        
        # Get bounding box for cropping
        bbox = self.bbox_cache[str(mask_path)]
        y_min, y_max, x_min, x_max = bbox[0], bbox[1], bbox[2], bbox[3]
        
        # Calculate center of vertebra
        y_center = (y_min + y_max) // 2
        x_center = (x_min + x_max) // 2
        
        # Crop around center
        half_crop = self.crop_size // 2
        y_start = max(0, y_center - half_crop)
        y_end = min(slice_2d.shape[0], y_center + half_crop)
        x_start = max(0, x_center - half_crop)
        x_end = min(slice_2d.shape[1], x_center + half_crop)
        
        cropped = slice_2d[y_start:y_end, x_start:x_end]
        
        # Pad if necessary to reach crop_size x crop_size
        if cropped.shape[0] < self.crop_size or cropped.shape[1] < self.crop_size:
            padded = np.zeros((self.crop_size, self.crop_size), dtype=np.float32)
            pad_y = (self.crop_size - cropped.shape[0]) // 2
            pad_x = (self.crop_size - cropped.shape[1]) // 2
            padded[pad_y:pad_y+cropped.shape[0], pad_x:pad_x+cropped.shape[1]] = cropped
            cropped = padded
        
        # Normalize to [0, 1]
        slice_min, slice_max = cropped.min(), cropped.max()
        if slice_max > slice_min:
            cropped = (cropped - slice_min) / (slice_max - slice_min)
        
        # Add channel dimension: (H, W) -> (1, H, W)
        cropped = np.expand_dims(cropped, axis=0)
        
        # Convert to tensor
        slice_tensor = torch.from_numpy(cropped)
        
        if self.transform:
            slice_tensor = self.transform(slice_tensor)
        
        return slice_tensor, torch.tensor(label, dtype=torch.long)

## 5. Create DataLoaders

In [ ]:
# Create full dataset WITH mask-based localization
full_dataset = VertebraSliceDatasetLocalized(
    ct_folder=CT_FOLDER,
    mask_folder=MASK_FOLDER,
    labels_dict=labels,
    num_slices=NUM_SLICES,
    crop_size=CROP_SIZE,
    transform=None
)

# Split into train/val
train_size = int((1 - VAL_SPLIT) * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"\nTrain samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

## 6. Visualize: Full Spine vs Cropped Vertebra

In [ ]:
# Visualize: Full spine vs Cropped vertebra (localization demo)
sample_ct_path = list(Path(CT_FOLDER).glob('*.nii.gz'))[0]
sample_mask_path = Path(MASK_FOLDER) / sample_ct_path.name

ct_vol = nib.load(str(sample_ct_path)).get_fdata()
mask_vol = nib.load(str(sample_mask_path)).get_fdata()

# Get center slice
z_center = ct_vol.shape[2] // 2

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Full spine
axes[0].imshow(ct_vol[:, :, z_center].T, cmap='gray', origin='lower')
axes[0].set_title('Full Straightened Spine (CT)')
axes[0].axis('off')

# Mask overlay
axes[1].imshow(ct_vol[:, :, z_center].T, cmap='gray', origin='lower')
axes[1].imshow(mask_vol[:, :, z_center].T, cmap='Reds', alpha=0.4, origin='lower')
axes[1].set_title('With Target Vertebra Mask (mask_2d)')
axes[1].axis('off')

# Find bbox and show cropped
nonzero = np.where(mask_vol > 0)
y_min, y_max = nonzero[0].min(), nonzero[0].max()
x_min, x_max = nonzero[1].min(), nonzero[1].max()
y_center_v, x_center_v = (y_min + y_max) // 2, (x_min + x_max) // 2

half = CROP_SIZE // 2
cropped = ct_vol[y_center_v-half:y_center_v+half, x_center_v-half:x_center_v+half, z_center]
axes[2].imshow(cropped.T, cmap='gray', origin='lower')
axes[2].set_title(f'Cropped Target Vertebra ({CROP_SIZE}x{CROP_SIZE})')
axes[2].axis('off')

plt.suptitle(f'Localization Demo: {sample_ct_path.name}', fontsize=14)
plt.tight_layout()
plt.show()

print(f"File: {sample_ct_path.name}")
print(f"Full spine shape: {ct_vol.shape}")
print(f"Vertebra bbox: y=[{y_min}:{y_max}], x=[{x_min}:{x_max}]")
print(f"Cropped region: {CROP_SIZE}x{CROP_SIZE} centered on target vertebra")

In [ ]:
# Get a batch and visualize cropped samples
images, labels_batch = next(iter(train_loader))
print(f"Batch shape: {images.shape}")  # Should be (batch, 1, 128, 128)
print(f"Labels: {labels_batch[:8]}")

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    if i < len(images):
        ax.imshow(images[i, 0].numpy(), cmap='gray')
        label_text = 'Healthy' if labels_batch[i] == 0 else 'Fractured'
        ax.set_title(f'{label_text} ({labels_batch[i].item()})')
        ax.axis('off')
plt.suptitle(f'Sample Training Images (Cropped {CROP_SIZE}x{CROP_SIZE} - Single Vertebra)')
plt.tight_layout()
plt.show()

## 7. Model Definition

**IMPORTANT**: Must match the architecture in `grad_CAM_3d_sagittal.py`:
```python
model = SEresnet50(spatial_dims=2, in_channels=1, num_classes=2)
model = torch.nn.DataParallel(model).cuda()
```

In [ ]:
def create_model():
    """Create SEResNet50 model matching grad_CAM requirements."""
    model = SEResNet50(
        spatial_dims=2,      # 2D images
        in_channels=1,       # Grayscale CT
        num_classes=2        # Binary: healthy/fractured
    )
    return model

model = create_model()
model = model.to(device)

# Print model summary
print(f"Model: SEResNet50")
print(f"Input size: 1 x {CROP_SIZE} x {CROP_SIZE}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 8. Loss Function and Optimizer

In [ ]:
# Calculate class weights for imbalanced data
all_labels = [l for _, _, _, l in full_dataset.samples]
n_healthy = sum(1 for l in all_labels if l == 0)
n_fractured = sum(1 for l in all_labels if l == 1)

# Inverse frequency weighting
if n_fractured > 0 and n_healthy > 0:
    weight_healthy = len(all_labels) / (2 * n_healthy)
    weight_fractured = len(all_labels) / (2 * n_fractured)
    class_weights = torch.tensor([weight_healthy, weight_fractured], dtype=torch.float32).to(device)
else:
    class_weights = None

print(f"Class weights: {class_weights}")

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

## 9. Training Functions

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, epoch_acc


def validate(model, loader, criterion, device):
    """Validate the model."""
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validating'):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(loader)
    
    # Metrics
    metrics = {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, zero_division=0),
        'recall': recall_score(all_labels, all_preds, zero_division=0),
        'f1': f1_score(all_labels, all_preds, zero_division=0)
    }
    
    return epoch_loss, metrics, all_preds, all_labels

## 10. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [],
    'val_f1': [], 'val_precision': [], 'val_recall': []
}

best_f1 = 0.0
best_epoch = 0

print(f"Starting training for {NUM_EPOCHS} epochs...")
print(f"APPROACH: WITH MASK LOCALIZATION ({CROP_SIZE}x{CROP_SIZE} crops)")
print("="*60)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_metrics, val_preds, val_labels = validate(model, val_loader, criterion, device)
    
    # Update scheduler
    scheduler.step()
    
    # Log
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_metrics['accuracy'])
    history['val_f1'].append(val_metrics['f1'])
    history['val_precision'].append(val_metrics['precision'])
    history['val_recall'].append(val_metrics['recall'])
    
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_metrics['accuracy']:.4f}")
    print(f"  Val F1: {val_metrics['f1']:.4f} | Precision: {val_metrics['precision']:.4f} | Recall: {val_metrics['recall']:.4f}")
    
    # Save best model
    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        best_epoch = epoch + 1
        
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_f1': best_f1,
            'history': history,
            'crop_size': CROP_SIZE,
            'approach': 'localized'
        }
        torch.save(checkpoint, os.path.join(CHECKPOINT_FOLDER, 'best_model_localized.tar'))
        print(f"  ✓ Saved best model (F1: {best_f1:.4f})")
    
    # Save periodic checkpoint
    if (epoch + 1) % 10 == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'history': history,
            'crop_size': CROP_SIZE,
            'approach': 'localized'
        }
        torch.save(checkpoint, os.path.join(CHECKPOINT_FOLDER, f'checkpoint_epoch_{epoch+1}.tar'))

print("\n" + "="*60)
print(f"Training complete!")
print(f"Best F1: {best_f1:.4f} at epoch {best_epoch}")
print(f"Checkpoints saved to: {CHECKPOINT_FOLDER}")

## 11. Training Curves

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['val_acc'], label='Val')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True)

# F1, Precision, Recall
axes[2].plot(history['val_f1'], label='F1')
axes[2].plot(history['val_precision'], label='Precision')
axes[2].plot(history['val_recall'], label='Recall')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Score')
axes[2].set_title('Validation Metrics')
axes[2].legend()
axes[2].grid(True)

plt.suptitle('Training Curves (WITH Localization)', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_FOLDER, 'training_curves_localized.png'), dpi=150)
plt.show()

## 12. Final Evaluation

In [ ]:
# Load best model
checkpoint = torch.load(os.path.join(CHECKPOINT_FOLDER, 'best_model_localized.tar'))
model.load_state_dict(checkpoint['state_dict'])
print(f"Loaded best model from epoch {checkpoint['epoch']}")

# Final validation
val_loss, val_metrics, val_preds, val_labels = validate(model, val_loader, criterion, device)

print("\n" + "="*60)
print("FINAL EVALUATION RESULTS (WITH LOCALIZATION)")
print("="*60)
print(f"Accuracy:  {val_metrics['accuracy']:.4f}")
print(f"Precision: {val_metrics['precision']:.4f}")
print(f"Recall:    {val_metrics['recall']:.4f}")
print(f"F1 Score:  {val_metrics['f1']:.4f}")

# Confusion Matrix
cm = confusion_matrix(val_labels, val_preds)
print(f"\nConfusion Matrix:")
print(f"              Pred 0   Pred 1")
print(f"Actual 0:     {cm[0,0]:5d}    {cm[0,1]:5d}")
print(f"Actual 1:     {cm[1,0]:5d}    {cm[1,1]:5d}")

# Classification Report
print("\nClassification Report:")
print(classification_report(val_labels, val_preds, target_names=['Healthy', 'Fractured']))

## 13. Save Model for Grad-CAM

**NOTE**: For Grad-CAM inference with this localized model, you'll need to:
1. Load the mask_2d for each CT file
2. Crop the CT around the target vertebra
3. Run inference on the cropped image
4. Map the heatmap back to the original coordinates

In [ ]:
# Wrap model in DataParallel to match grad_CAM expectations
model_dp = nn.DataParallel(model)

# Save checkpoint in expected format
gradcam_checkpoint = {
    'epoch': checkpoint['epoch'],
    'state_dict': model_dp.state_dict(),  # DataParallel state dict
    'best_f1': best_f1,
    'metrics': val_metrics,
    'crop_size': CROP_SIZE,
    'approach': 'localized',
    'note': 'This model expects cropped 128x128 input centered on target vertebra'
}

save_path = os.path.join(CHECKPOINT_FOLDER, 'seresnet50_classifier_localized.tar')
torch.save(gradcam_checkpoint, save_path)
print(f"\nGrad-CAM compatible checkpoint saved to:")
print(f"  {save_path}")

print("\n" + "="*60)
print("IMPORTANT: GRAD-CAM USAGE")
print("="*60)
print(f"This model expects {CROP_SIZE}x{CROP_SIZE} cropped input!")
print(f"You need to modify grad_CAM_3d_sagittal.py to:")
print(f"  1. Load mask_2d for each CT file")
print(f"  2. Find bounding box of target vertebra")
print(f"  3. Crop CT to {CROP_SIZE}x{CROP_SIZE} around center")
print(f"  4. Generate heatmap on cropped region")
print(f"  5. Map heatmap back to original coordinates")

## 14. Confusion Matrix Visualization

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Healthy', 'Fractured'],
            yticklabels=['Healthy', 'Fractured'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix (WITH Localization)\nAccuracy: {val_metrics["accuracy"]:.2%}, F1: {val_metrics["f1"]:.4f}')
plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_FOLDER, 'confusion_matrix_localized.png'), dpi=150)
plt.show()

## Summary: Approach Comparison

### This Notebook (WITH Localization)
- **Input**: Cropped 128×128 region around target vertebra (using mask_2d)
- **Pros**: Model sees ONLY target vertebra, cleaner learning signal
- **Cons**: Requires mask at inference, need to modify Grad-CAM script

### Other Notebook (WITHOUT Localization)
- **Input**: Full 256×256 slice (target centered, neighbors visible)
- **Pros**: Simpler pipeline, works with existing Grad-CAM script
- **Cons**: Model might learn features from neighboring vertebrae

### Compare Results
Run both notebooks and compare:
1. Validation F1 scores
2. Confusion matrices
3. Grad-CAM heatmaps (where does the model focus?)